In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
import yaml

# 1. Carregar os dados (que já estão com One-Hot Encoding do notebook 1)
with open("../config.yaml", "r") as file:
    config = yaml.safe_load(file)

train_df = pd.read_csv(config['output_data']['train_file'])
test_df = pd.read_csv(config['output_data']['test_file'])

# 2. Separar Features (X) e Target (y)
# Remover colunas de texto/descrição se houverem sobrado, manter apenas numéricas e one-hot
features_to_drop = ['pairing_quality', 'quality_label', 'description'] 
# Nota: certifique-se de remover IDs brutos se for usar One-Hot, ou remover One-Hot se for usar IDs (CatBoost usa IDs bem).
# Assumindo o uso do One-Hot gerado no notebook 1:

X_train = train_df.drop(columns=features_to_drop, errors='ignore')
y_train = train_df['pairing_quality']

X_test = test_df.drop(columns=features_to_drop, errors='ignore')
y_test = test_df['pairing_quality']

# 3. Treinar XGBoost Regressor
model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=500,     # Mais árvores
    learning_rate=0.05,   # Aprendizado mais lento e preciso
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

model.fit(X_train, y_train)

# 4. Prever e Avaliar
predictions = model.predict(X_test)

rmse = sqrt(mean_squared_error(y_test, predictions))
mae = mean_absolute_error(y_test, predictions)

print(f"📊 XGBoost Evaluation:")
print(f"  RMSE: {rmse:.4f}")
print(f"  MAE : {mae:.4f}")

📊 XGBoost Evaluation:
  RMSE: 1.4823
  MAE : 1.2494
